In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torchvision
import math
from sklearn.preprocessing import StandardScaler
from thermography_dataset import ThermDataset
from torch.utils.data import DataLoader

In [2]:
dataset = ThermDataset(fp='combined_data.xlsx', noise_scale=0.001)

In [3]:
dataset.wavelengths

Index([4.75e-06,  4.8e-06, 4.85e-06,  4.9e-06, 4.95e-06,    5e-06, 5.05e-06,
        5.1e-06, 5.15e-06,  5.2e-06, 5.25e-06,  5.3e-06, 5.35e-06,  5.4e-06,
       5.45e-06,  5.5e-06, 5.55e-06,  5.6e-06, 5.65e-06,  5.7e-06, 5.75e-06,
        5.8e-06, 5.85e-06,  5.9e-06, 5.95e-06,    6e-06, 6.05e-06,  6.1e-06,
       6.15e-06,  6.2e-06, 6.25e-06,  6.3e-06, 6.35e-06,  6.4e-06, 6.45e-06,
        6.5e-06, 6.55e-06,  6.6e-06, 6.65e-06,  6.7e-06, 6.75e-06,  6.8e-06,
       6.85e-06,  6.9e-06, 6.95e-06,    7e-06, 7.05e-06,  7.1e-06, 7.15e-06,
        7.2e-06, 7.25e-06,  7.3e-06, 7.35e-06,  7.4e-06, 7.45e-06,  7.5e-06,
       7.55e-06,  7.6e-06, 7.65e-06,  7.7e-06, 7.75e-06,  7.8e-06, 7.85e-06,
        7.9e-06, 7.95e-06,    8e-06],
      dtype='object')

In [4]:
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers=2)

In [5]:
dataiter = iter(dataloader)
data = next(dataiter)
features, labels = data
print(features, labels)

tensor([[355.0373, 314.4801, 323.0569, 311.7884, 317.3070, 326.9681, 329.5618,
         322.2959, 331.5455, 331.1420, 344.9434],
        [351.1057, 318.8794, 330.9963, 326.2987, 340.7686, 350.9916, 349.4833,
         351.1031, 358.0569, 353.7625, 362.3861],
        [339.6719, 325.2686, 321.9301, 327.3522, 327.8385, 330.2975, 349.8128,
         340.6671, 326.3076, 326.8924, 294.9841],
        [344.0456, 326.4657, 328.9952, 316.9877, 319.2746, 325.1846, 337.8809,
         324.3984, 336.4857, 335.3642, 344.4449]]) tensor([[0.3920, 0.4481, 0.4947, 0.5372, 0.5777, 0.6165, 0.6579, 0.7011, 0.7447,
         0.7866, 0.8306, 0.8787, 0.9211, 0.9670, 1.0158, 1.0572, 1.1070, 1.1562,
         1.2077, 1.2560, 1.3090, 1.3578, 1.4135, 1.4690, 1.5196, 1.5702, 1.6296,
         1.6833, 1.7419, 1.8029, 1.8612, 1.9236, 1.9865, 2.0554, 2.1181, 2.1848,
         2.2513, 2.3186, 2.3870, 2.4562, 2.5228, 2.5904, 2.6646, 2.7359, 2.8131,
         2.8912, 2.9725, 3.0489, 3.1267, 3.1999, 3.2661, 3.3396, 3.4057, 3.476

In [6]:
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)

In [7]:
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        if i+1 == 5:
            print(labels.shape)
            scaler = StandardScaler()
            scaled = torch.tensor(scaler.fit_transform(labels), dtype=torch.float32)
            print(scaled.shape)


torch.Size([4, 66])
torch.Size([4, 66])
torch.Size([4, 66])
torch.Size([4, 66])


In [9]:
[range(1,11)]

[range(1, 11)]